<a href="https://colab.research.google.com/github/juanandres-montero/DataAnalysis/blob/main/Obtener_datos_del_TSE_por_cedulas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown # Instalar requerimientos
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver_manager
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from google.colab import files

In [ ]:
import time
from google.colab import files
def get_info(cedula):
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=options)
  driver.get('https://servicioselectorales.tse.go.cr/chc/consulta_cedula.aspx')
  # Enter the cedula number
  box = driver.find_element('xpath','//*[@id="txtcedula"]')
  box.send_keys(cedula)
  driver.find_element('xpath','//*[@id="btnConsultaCedula"]').click()
  time.sleep(1)
  driver.find_element('xpath','//*[@id="LinkButton11"]').click()
  time.sleep(1)
  table = driver.find_element('xpath', '//*[@id="form1"]/table[2]')
  rows = table.find_elements('xpath','//*[@id="form1"]/table[2]/tbody/tr')
  df = pd.read_html(table.get_attribute('outerHTML'))
  driver.find_element('xpath','//*[@id="ImageConsultaCedula"]').click()
  time.sleep(1)
  return df

def format(df):
  df_transposed = df.transpose()
  f0 = df_transposed.iloc[[0]]
  f1 = df_transposed.iloc[[1]]
  f2 = df_transposed.iloc[[2]]
  f3 = df_transposed.iloc[[3]]
  df0 = pd.concat([f0, f1], axis=0)
  df1 = pd.concat([f2, f3], axis=0)
  df1 = df1.reset_index()
  new_df = pd.concat([df0, df1], axis=1)
  new_df = new_df.drop(columns=['index'])
  new_df.columns = new_df.iloc[0]
  new_df = new_df.drop(0)
  new_df = new_df.dropna(axis=1, how='all')
  return new_df
#@markdown - Ingrese los números de cédula separados por comas sin espacios
def main():
  Numeros_de_cedulas = "105600795" #@param {type:"string"}
  cedulas = (Numeros_de_cedulas).split(',')
  df = pd.DataFrame()
  data = pd.DataFrame()
  save = pd.DataFrame()
  for cedula in cedulas:
    try:
      df = df.append(get_info(cedula))
      data = data.append(format(df))
      save = save.append(data)

      data = pd.DataFrame()
      df = pd.DataFrame()
    except:
      print('Error: Invalid cedula number')
    # Reformatear y organizar los datos
  print(save)
  save.to_excel("data_export.xlsx", sheet_name="Sheet1")
  files.download("data_export.xlsx")

if __name__ == '__main__':
  main()
#@markdown Los datos del TSE de las cédulas se exportarán automáticamente en formato excel.

0 Número de Cédula:                   Nombre: Primer Apellido:  \
1         105600795  RODRIGO ALBERTO DE JESUS           CHAVES   

0 Segundo Apellido: Fecha de Nacimiento:        Lugar de Nacimiento:  \
1            ROBLES           10/06/1961  EL CARMEN CENTRAL SAN JOSE   

0  Nacionalidad:               Hijo/a de: Identificación:  \
1  COSTARRICENSE  RODRIGO CHAVES ARGUEDAS               0   

0                     Y: Identificación: Empadronado/a: Fallecido/a:  \
1  ALICIA ROBLES JIMENEZ       102140104             SI           NO   

0 Marginal:      Sexo:  
1        NO  MASCULINO  


<ipython-input-125-bbf3085306bb>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_info(cedula))
<ipython-input-125-bbf3085306bb>:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(format(df))
<ipython-input-125-bbf3085306bb>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  save = save.append(data)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>